In [ ]:
%%time 

import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable


# Hyper Parameters 
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

# MNIST Dataset 
train_dataset = dsets.MNIST(root='../data', 
                            train=True, 
                            transform=transforms.ToTensor(),  
                            download=True)

test_dataset = dsets.MNIST(root='../data', 
                           train=False, 
                           transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

# Neural Network Model (1 hidden layer)
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out
    
net = Net(input_size, hidden_size, num_classes)
net#.cuda()   
    
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)  

# Train the Model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Convert torch tensor to Variable
        images = Variable(images.view(-1, 28*28))#.cuda()
        labels = Variable(labels)#.cuda()
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()  # zero the gradient buffer
        outputs = net(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.item()))

# Test the Model
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, 28*28))#.cuda()
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted.cpu() == labels).sum()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

# Save the Model
torch.save(net.state_dict(), 'model.pkl')

Epoch [1/5], Step [100/600], Loss: 0.3966
Epoch [1/5], Step [200/600], Loss: 0.2649
Epoch [1/5], Step [300/600], Loss: 0.3568
Epoch [1/5], Step [400/600], Loss: 0.2293
Epoch [1/5], Step [500/600], Loss: 0.2534
Epoch [1/5], Step [600/600], Loss: 0.1600
Epoch [2/5], Step [100/600], Loss: 0.2174
Epoch [2/5], Step [200/600], Loss: 0.1355
Epoch [2/5], Step [300/600], Loss: 0.1206
Epoch [2/5], Step [400/600], Loss: 0.0584
Epoch [2/5], Step [500/600], Loss: 0.1340
Epoch [2/5], Step [600/600], Loss: 0.1684
Epoch [3/5], Step [100/600], Loss: 0.0444
Epoch [3/5], Step [200/600], Loss: 0.0445
Epoch [3/5], Step [300/600], Loss: 0.1405
Epoch [3/5], Step [400/600], Loss: 0.1445
Epoch [3/5], Step [500/600], Loss: 0.1796
Epoch [3/5], Step [600/600], Loss: 0.0368
Epoch [4/5], Step [100/600], Loss: 0.0523
Epoch [4/5], Step [200/600], Loss: 0.0494
Epoch [4/5], Step [300/600], Loss: 0.0241
Epoch [4/5], Step [400/600], Loss: 0.0747
Epoch [4/5], Step [500/600], Loss: 0.0971
Epoch [4/5], Step [600/600], Loss: